In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
# https://raw.githubusercontent.com/leaderj1001/Stand-Alone-Self-Attention/master/attention.py
import math


class AttentionConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, groups=1, bias=False):
        super(AttentionConv, self).__init__()
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.groups = groups

        assert self.out_channels % self.groups == 0, "out_channels should be divided by groups. (example: out_channels: 40, groups: 4)"

        self.rel_h = nn.Parameter(torch.randn(out_channels // 2, 1, 1, kernel_size, 1), requires_grad=True)
        self.rel_w = nn.Parameter(torch.randn(out_channels // 2, 1, 1, 1, kernel_size), requires_grad=True)

        self.key_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)
        self.query_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)
        self.value_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)

        self.reset_parameters()

    def forward(self, x):
        batch, channels, height, width = x.size()

        padded_x = F.pad(x, [self.padding, self.padding, self.padding, self.padding])
        q_out = self.query_conv(x)
        k_out = self.key_conv(padded_x)
        v_out = self.value_conv(padded_x)

        k_out = k_out.unfold(2, self.kernel_size, self.stride).unfold(3, self.kernel_size, self.stride)
        v_out = v_out.unfold(2, self.kernel_size, self.stride).unfold(3, self.kernel_size, self.stride)

        k_out_h, k_out_w = k_out.split(self.out_channels // 2, dim=1)
        k_out = torch.cat((k_out_h + self.rel_h, k_out_w + self.rel_w), dim=1)

        k_out = k_out.contiguous().view(batch, self.groups, self.out_channels // self.groups, height, width, -1)
        v_out = v_out.contiguous().view(batch, self.groups, self.out_channels // self.groups, height, width, -1)

        q_out = q_out.view(batch, self.groups, self.out_channels // self.groups, height, width, 1)

        out = q_out * k_out
        out = F.softmax(out, dim=-1)
        out = torch.einsum('bnchwk,bnchwk -> bnchw', out, v_out).view(batch, -1, height, width)

        return out

    def reset_parameters(self):
        init.kaiming_normal_(self.key_conv.weight, mode='fan_out', nonlinearity='relu')
        init.kaiming_normal_(self.value_conv.weight, mode='fan_out', nonlinearity='relu')
        init.kaiming_normal_(self.query_conv.weight, mode='fan_out', nonlinearity='relu')

        init.normal_(self.rel_h, 0, 1)
        init.normal_(self.rel_w, 0, 1)


class AttentionStem(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, groups=1, m=4, bias=False):
        super(AttentionStem, self).__init__()
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.groups = groups
        self.m = m

        assert self.out_channels % self.groups == 0, "out_channels should be divided by groups. (example: out_channels: 40, groups: 4)"

        self.emb_a = nn.Parameter(torch.randn(out_channels // groups, kernel_size), requires_grad=True)
        self.emb_b = nn.Parameter(torch.randn(out_channels // groups, kernel_size), requires_grad=True)
        self.emb_mix = nn.Parameter(torch.randn(m, out_channels // groups), requires_grad=True)

        self.key_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)
        self.query_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)
        self.value_conv = nn.ModuleList([nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias) for _ in range(m)])

        self.reset_parameters()

    def forward(self, x):
        batch, channels, height, width = x.size()

        padded_x = F.pad(x, [self.padding, self.padding, self.padding, self.padding])

        q_out = self.query_conv(x)
        k_out = self.key_conv(padded_x)
        v_out = torch.stack([self.value_conv[_](padded_x) for _ in range(self.m)], dim=0)

        k_out = k_out.unfold(2, self.kernel_size, self.stride).unfold(3, self.kernel_size, self.stride)
        v_out = v_out.unfold(3, self.kernel_size, self.stride).unfold(4, self.kernel_size, self.stride)

        k_out = k_out[:, :, :height, :width, :, :]
        v_out = v_out[:, :, :, :height, :width, :, :]

        emb_logit_a = torch.einsum('mc,ca->ma', self.emb_mix, self.emb_a)
        emb_logit_b = torch.einsum('mc,cb->mb', self.emb_mix, self.emb_b)
        emb = emb_logit_a.unsqueeze(2) + emb_logit_b.unsqueeze(1)
        emb = F.softmax(emb.view(self.m, -1), dim=0).view(self.m, 1, 1, 1, 1, self.kernel_size, self.kernel_size)

        v_out = emb * v_out

        k_out = k_out.contiguous().view(batch, self.groups, self.out_channels // self.groups, height, width, -1)
        v_out = v_out.contiguous().view(self.m, batch, self.groups, self.out_channels // self.groups, height, width, -1)
        v_out = torch.sum(v_out, dim=0).view(batch, self.groups, self.out_channels // self.groups, height, width, -1)

        q_out = q_out.view(batch, self.groups, self.out_channels // self.groups, height, width, 1)

        out = q_out * k_out
        out = F.softmax(out, dim=-1)
        out = torch.einsum('bnchwk,bnchwk->bnchw', out, v_out).view(batch, -1, height, width)

        return out

    def reset_parameters(self):
        init.kaiming_normal_(self.key_conv.weight, mode='fan_out', nonlinearity='relu')
        init.kaiming_normal_(self.query_conv.weight, mode='fan_out', nonlinearity='relu')
        for _ in self.value_conv:
            init.kaiming_normal_(_.weight, mode='fan_out', nonlinearity='relu')

        init.normal_(self.emb_a, 0, 1)
        init.normal_(self.emb_b, 0, 1)
        init.normal_(self.emb_mix, 0, 1)


# temp = torch.randn((2, 3, 32, 32))
# conv = AttentionConv(3, 16, kernel_size=3, padding=1)
# print(conv(temp).size())

In [2]:
import torch
import torch.nn as nn
from torchvision import models
from torch.hub import load_state_dict_from_url

from PIL import Image
import cv2
import numpy as np
from matplotlib import pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
# from torchsummary import summary

# https://raw.githubusercontent.com/spmallick/learnopencv/master/PyTorch-Fully-Convolutional-Image-Classification/FullyConvolutionalResnet18.py
# Define the architecture by modifying resnet. 
# Original code is here 
# https://github.com/pytorch/vision/blob/b2e95657cd5f389e3973212ba7ddbdcc751a7878/torchvision/models/resnet.py
class FullyConvolutionalResnet18(models.ResNet):
    def __init__(self, num_classes=1000, pretrained=False, **kwargs):

        super().__init__(block=models.resnet.BasicBlock, layers=[2, 2, 2, 2], num_classes=num_classes, **kwargs)
        if pretrained:
            state_dict = load_state_dict_from_url(models.resnet.model_urls["resnet18"], progress=True)
            self.load_state_dict(state_dict)

        self.avgpool = nn.AvgPool2d((3, 3))# change size; figure this out

        self.conv_final = AttentionConv(in_channels=self.fc.in_features, out_channels=num_classes, kernel_size=1)#nn.Conv2d
#         self.conv_final.weight.data.copy_(self.fc.weight.data.view(*self.fc.weight.data.shape, 1, 1))
#         self.conv_final.bias.data.copy_(self.fc.bias.data)
        self.use_conv=False
        
    def toggle_conv(self,use_conv=False):
        self.use_conv=use_conv

    def _forward_impl(self, x):
        # Standard forward for resnet18
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = self.conv_final(x)
        if not self.use_conv: 
            y = F.max_pool2d(x, kernel_size=x.size()[2:])[...,-1,-1]
            return y,x
        return x

In [3]:
import pandas as pd
df=pd.read_pickle("dataset.pkl")

In [4]:
df['complete']=(df['complete']>=1).astype(int)

In [5]:
df['complete']=df['complete'].map(lambda x: torch.tensor(x).long())

In [6]:
df['image']=df['image'].map(lambda x: x[0])

In [7]:
from sklearn.model_selection import train_test_split
data={}
data['train'],data['val']=train_test_split(df,train_size=0.7,stratify=df['complete'])

In [8]:
net=FullyConvolutionalResnet18(3)

In [9]:
# preds=net(df['image'].iloc[0])
# print(preds.shape)
# net.use_conv=True
# preds=net(df['image'].iloc[0])
# print(preds.shape)
# preds = torch.softmax(preds, dim=1)
# pred, class_idx = torch.max(preds, dim=1)

In [27]:
import time
import os
import sys
import seaborn as sns
from PathPretrain.schedulers import Scheduler
import torch
from torch import nn
from torchvision import models
from torchvision.models.mobilenet import MobileNetV2
from sklearn.metrics import classification_report
#from apex import amp
import copy
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from visdom import Visdom
from sklearn.utils.class_weight import compute_class_weight
import kornia.augmentation as K, kornia.geometry.transform as G

matplotlib.use('Agg')
sns.set()



class ModelTrainer:
    """Trainer for the neural network model that wraps it into a scikit-learn like interface.

    Parameters
    ----------
    model:nn.Module
            Deep learning pytorch model.
    n_epoch:int
            Number training epochs.
    validation_dataloader:DataLoader
            Dataloader of validation dataset.
    optimizer_opts:dict
            Options for optimizer.
    scheduler_opts:dict
            Options for learning rate scheduler.
    loss_fn:str
            String to call a particular loss function for model.
    reduction:str
            Mean or sum reduction of loss.
    num_train_batches:int
            Number of training batches for epoch.
    """

    def __init__(self, model, n_epoch=300, validation_dataloader=None, 
                 optimizer_opts=dict(name='adam', lr=1e-3, weight_decay=1e-4), 
                 scheduler_opts=dict(scheduler='warm_restarts', lr_scheduler_decay=0.5, T_max=10, eta_min=5e-8, T_mult=2), 
                 loss_fn='ce', reduction='mean', num_train_batches=None, opt_level='O1', 
                 checkpoints_dir='checkpoints',tensor_dataset=False,transforms=None,
                 accumulation=1,port=4444):

        self.model = model
        # self.amp_handle = amp.init(enabled=True)
        optimizers = {'adam': torch.optim.Adam, 'sgd': torch.optim.SGD}
        loss_functions = {'bce': nn.BCEWithLogitsLoss(reduction=reduction), 'ce': nn.CrossEntropyLoss(
            reduction=reduction), 'mse': nn.MSELoss(reduction=reduction), 'nll': nn.NLLLoss(reduction=reduction)}
        if 'name' not in list(optimizer_opts.keys()):
            optimizer_opts['name'] = 'adam'
        self.optimizer = optimizers[optimizer_opts.pop('name')](
            self.model.parameters(), **optimizer_opts)
        if False and torch.cuda.is_available():
            self.cuda = True
            self.model, self.optimizer = amp.initialize(
                self.model, self.optimizer, opt_level=opt_level)
        else:
            self.cuda = False
        self.scheduler = Scheduler(
            optimizer=self.optimizer, opts=scheduler_opts)
        self.n_epoch = n_epoch
        self.validation_dataloader = validation_dataloader
        self.loss_fn = loss_functions[loss_fn]
        self.loss_fn_name = loss_fn
        self.bce = (self.loss_fn_name == 'bce')
        self.sigmoid = nn.Sigmoid()
        self.original_loss_fn = copy.deepcopy(loss_functions[loss_fn])
        self.num_train_batches = num_train_batches
        self.val_loss_fn = copy.deepcopy(loss_functions[loss_fn])
        self.verbosity=0
        self.checkpoints_dir=checkpoints_dir
        self.tensor_dataset=tensor_dataset
        self.transforms=transforms
        self.accumulation=accumulation
        self.vis=Visdom(port=port)
        self.vis_every=100

    def save_checkpoint(self,model,epoch):
        os.makedirs(self.checkpoints_dir,exist_ok=True)
        torch.save(model,os.path.join(self.checkpoints_dir,f"{epoch}.checkpoint.pth"))

    def calc_loss(self, y_pred, y_true):
        """Calculates loss supplied in init statement and modified by reweighting.

        Parameters
        ----------
        y_pred:tensor
                Predictions.
        y_true:tensor
                True values.

        Returns
        -------
        loss

        """

        return self.loss_fn(y_pred, y_true)

    def calc_val_loss(self, y_pred, y_true):
        """Calculates loss supplied in init statement on validation set.

        Parameters
        ----------
        y_pred:tensor
                Predictions.
        y_true:tensor
                True values.

        Returns
        -------
        val_loss

        """

        return self.val_loss_fn(y_pred, y_true)

    def reset_loss_fn(self):
        """Resets loss to original specified loss."""
        self.loss_fn = self.original_loss_fn

    def add_class_balance_loss(self, y, custom_weights=''):
        """Updates loss function to handle class imbalance by weighting inverse to class appearance.

        Parameters
        ----------
        dataset:DynamicImageDataset
                Dataset to balance by.

        """
        self.class_weights = compute_class_weight('balanced',np.unique(y),y)#dataset.get_class_weights() if not custom_weights else np.array(
            #list(map(float, custom_weights.split(','))))
        if custom_weights:
            self.class_weights = self.class_weights / sum(self.class_weights)
        print('Weights:', self.class_weights)
        self.original_loss_fn = copy.deepcopy(self.loss_fn)
        weight = torch.tensor(self.class_weights, dtype=torch.float)
        if torch.cuda.is_available():
            weight = weight.cuda()
        if self.loss_fn_name == 'ce':
            self.loss_fn = nn.CrossEntropyLoss(weight=weight)
        elif self.loss_fn_name == 'nll':
            self.loss_fn = nn.NLLLoss(weight=weight)
        else:  # modify below for multi-target
            self.loss_fn = lambda y_pred, y_true: sum([self.class_weights[i] * self.original_loss_fn(
                y_pred[y_true == i], y_true[y_true == i]) if sum(y_true == i) else 0. for i in range(2)])

    def calc_best_confusion(self, y_pred, y_true):
        """Calculate confusion matrix on validation set for classification/segmentation tasks, optimize threshold where positive.

        Parameters
        ----------
        y_pred:array
                Predictions.
        y_true:array
                Ground truth.

        Returns
        -------
        float
                Optimized threshold to use on test set.
        dataframe
                Confusion matrix.

        """
        fpr, tpr, thresholds = roc_curve(y_true, y_pred)
        threshold = thresholds[np.argmin(
            np.sum((np.array([0, 1]) - np.vstack((fpr, tpr)).T)**2, axis=1)**.5)]
        y_pred = (y_pred > threshold).astype(int)
        return threshold, pd.DataFrame(confusion_matrix(y_true, y_pred), index=['F', 'T'], columns=['-', '+']).iloc[::-1, ::-1].T

    def loss_backward(self, loss):
        """Backprop using mixed precision for added speed boost.

        Parameters
        ----------
        loss:loss
                Torch loss calculated.

        """
        # with self.amp_handle.scale_loss(loss, self.optimizer) as scaled_loss:
        # 	scaled_loss.backward()
        # loss.backward()
        if self.cuda:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()

    # @pysnooper.snoop('train_loop.log')
    def train_loop(self, epoch, train_dataloader):
        """One training epoch, calculate predictions, loss, backpropagate.

        Parameters
        ----------
        epoch:int
                Current epoch.
        train_dataloader:DataLoader
                Training data.

        Returns
        -------
        float
                Training loss for epoch

        """
        self.model.train(True)
        running_loss = 0.
        n_batch = len(
            train_dataloader.dataset) // train_dataloader.batch_size if self.num_train_batches == None else self.num_train_batches
        n_total_batches=0
        n_batches_backward=self.accumulation
        for i, (X, y_true) in enumerate(train_dataloader):
            n_total_batches+=1
            starttime = time.time()
            if i == n_batch:
                break
            # X = Variable(batch[0], requires_grad=True)
            # y_true = Variable(batch[1])
            

            if torch.cuda.is_available():
                X = X.cuda()
                y_true = y_true.cuda()

            if i%self.vis_every==0: 
                self.vis.close("img")
                self.vis.image(G.resize(X,(np.array(X.shape[-2:])/4).astype(int).tolist())[0],win="img")
            if self.tensor_dataset:
                X=self.transforms['train'](X)
                
            
            
            y_pred,y_map = self.model(X)
            
            if i%self.vis_every==0:
                self.vis.close("pred")
                self.vis.image(G.resize(y_map.detach().cpu(),(np.array(y_map.shape[-2:])*20).tolist())[0,0,...],win="pred")#,interpolation="bicubic"
                self.vis.text(f"Incomplete: {np.abs(1-y_true[0].item())}",win="txt")#[::-1]
                self.vis.text(f"Incomplete score: {F.softmax(y_pred,dim=1).flatten()[0].item()}",win="txt2")
                
            
            # y_true=y_true.argmax(dim=1)


            loss = self.calc_loss(y_pred, y_true)  # .view(-1,1)
            train_loss = loss.item()
            running_loss += train_loss
#             self.optimizer.zero_grad()
            self.loss_backward(loss)  # loss.backward()
            if n_total_batches%n_batches_backward==0 or (i==len(train_dataloader.dataset)-1):
                self.optimizer.step()
                self.optimizer.zero_grad()
#             self.optimizer.step()
            torch.cuda.empty_cache()
            endtime = time.time()
            if self.verbosity >=1:
                print("Epoch {}[{}/{}] Time:{}, Train Loss:{}".format(epoch,
                                                                  i, n_batch, round(endtime - starttime, 3), train_loss))
        self.scheduler.step()
        running_loss /= n_batch
        return running_loss

    def val_loop(self, epoch, val_dataloader, print_val_confusion=True, save_predictions=True):
        """Calculate loss over validation set.

        Parameters
        ----------
        epoch:int
                Current epoch.
        val_dataloader:DataLoader
                Validation iterator.
        print_val_confusion:bool
                Calculate confusion matrix and plot.
        save_predictions:int
                Print validation results.

        Returns
        -------
        float
                Validation loss for epoch.
        """
        self.model.train(False)
        n_batch = len(val_dataloader.dataset) // val_dataloader.batch_size
        running_loss = 0.
        Y = {'pred': [], 'true': []}
        with torch.no_grad():
            for i, (X, y_true) in enumerate(val_dataloader):
                # X = Variable(batch[0], requires_grad=True)
                # y_true = Variable(batch[1])
                if i%self.vis_every==0:
                    self.vis.close("img")
                    self.vis.image(G.resize(X,(np.array(X.shape[-2:])/4).astype(int).tolist())[0],win="img")
                
                if torch.cuda.is_available():
                    X = X.cuda()
                    y_true = y_true.cuda()

                if self.tensor_dataset:
                    X=self.transforms['val'](X)

                y_pred,y_map = self.model(X)
                
                if i%self.vis_every==0:
                    self.vis.close("pred")
                    self.vis.image(G.resize(y_map.detach().cpu(),(np.array(y_map.shape[-2:])*20).tolist())[0,0,...],win="pred")#,interpolation="bicubic"
                    self.vis.text(f"Incomplete: {np.abs(1-y_true[0].item())}",win="txt")#[::-1]
                    self.vis.text(f"Incomplete score: {F.softmax(y_pred,dim=1).flatten()[0].item()}",win="txt2")
                
                # y_true=y_true.argmax(dim=1)
                if save_predictions:
                    Y['true'].append(
                        y_true.detach().cpu().numpy().astype(int).flatten())
                    y_pred_numpy = ((y_pred if not self.bce else self.sigmoid(
                        y_pred)).detach().cpu().numpy()).astype(float)
                    if self.loss_fn_name == 'ce':
                        y_pred_numpy = y_pred_numpy.argmax(axis=1)
                    Y['pred'].append(y_pred_numpy.flatten())
                loss = self.calc_val_loss(y_pred, y_true)  # .view(-1,1)
                val_loss = loss.item()
                running_loss += val_loss
                if self.verbosity >=1:
                    print("Epoch {}[{}/{}] Val Loss:{}".format(epoch, i, n_batch, val_loss))
        if print_val_confusion and save_predictions:
            y_pred, y_true = np.hstack(Y['pred']), np.hstack(Y['true'])
            print(classification_report(y_true, y_pred))
        running_loss /= n_batch
        return running_loss

    # @pysnooper.snoop("test_loop.log")
    def test_loop(self, test_dataloader):
        """Calculate final predictions on loss.

        Parameters
        ----------
        test_dataloader:DataLoader
                Test dataset.

        Returns
        -------
        array
                Predictions or embeddings.
        """
        # self.model.train(False) KEEP DROPOUT? and BATCH NORM??
        self.model.eval()
        y_pred = []
        Y_true = []
        running_loss = 0.
        with torch.no_grad():
            for i, (X, y_true) in enumerate(test_dataloader):
                #X = Variable(batch[0],requires_grad=False)
                if torch.cuda.is_available():
                    X = X.cuda()
                    y_true = y_true.cuda()

                prediction = self.model(X)
                y_pred.append(prediction.detach().cpu().numpy())
                Y_true.append(y_true.detach().cpu().numpy())
        y_pred = np.concatenate(y_pred, axis=0)  # torch.cat(y_pred,0)
        y_true = np.concatenate(Y_true, axis=0).flatten()
        return y_pred,y_true

    def fit(self, train_dataloader, verbose=False, print_every=10, save_model=True, plot_training_curves=False, plot_save_file=None, print_val_confusion=True, save_val_predictions=True):
        """Fits the segmentation or classification model to the patches, saving the model with the lowest validation score.

        Parameters
        ----------
        train_dataloader:DataLoader
                Training dataset.
        verbose:bool
                Print training and validation loss?
        print_every:int
                Number of epochs until print?
        save_model:bool
                Whether to save model when reaching lowest validation loss.
        plot_training_curves:bool
                Plot training curves over epochs.
        plot_save_file:str
                File to save training curves.
        print_val_confusion:bool
                Print validation confusion matrix.
        save_val_predictions:bool
                Print validation results.

        Returns
        -------
        self
                Trainer.
        float
                Minimum val loss.
        int
                Best validation epoch with lowest loss.

        """
        # choose model with best f1
        self.train_losses = []
        self.val_losses = []
        if verbose:
            self.verbosity+=1
        for epoch in range(self.n_epoch):
            start_time = time.time()
            train_loss = self.train_loop(epoch, train_dataloader)
            current_time = time.time()
            train_time = current_time - start_time
            self.train_losses.append(train_loss)
            val_loss = self.val_loop(epoch, self.validation_dataloader,
                                     print_val_confusion=print_val_confusion, save_predictions=save_val_predictions)
            val_time = time.time() - current_time
            self.val_losses.append(val_loss)
            if True:#verbose and not (epoch % print_every):
                if plot_training_curves:
                    self.plot_train_val_curves(plot_save_file)
                print("Epoch {}: Train Loss {}, Val Loss {}, Train Time {}, Val Time {}".format(
                    epoch, train_loss, val_loss, train_time, val_time))
            if val_loss <= min(self.val_losses) and save_model:
                print("New best model at epoch {}".format(epoch))
                min_val_loss = val_loss
                best_epoch = epoch
                best_model_state_dict = copy.deepcopy(self.model.state_dict())
                self.save_checkpoint(best_model_state_dict,epoch)
        if save_model:
            print("Saving best model at epoch {}".format(best_epoch))
            self.model.load_state_dict(best_model_state_dict)
        return self, min_val_loss, best_epoch

    def plot_train_val_curves(self, save_file=None):
        """Plots training and validation curves.

        Parameters
        ----------
        save_file:str
                File to save to.

        """
        plt.figure()
        sns.lineplot('epoch', 'value', hue='variable',
                     data=pd.DataFrame(np.vstack((np.arange(len(self.train_losses)), self.train_losses, self.val_losses)).T,
                                       columns=['epoch', 'train', 'val']).melt(id_vars=['epoch'], value_vars=['train', 'val']))
        if save_file is not None:
            plt.savefig(save_file, dpi=300)

    def predict(self, test_dataloader):
        """Make classification segmentation predictions on testing data.

        Parameters
        ----------
        test_dataloader:DataLoader
                Test data.

        Returns
        -------
        array
                Predictions.

        """
        y_pred,y_true = self.test_loop(test_dataloader)
        return y_pred,y_true

    def fit_predict(self, train_dataloader, test_dataloader):
        """Fit model to training data and make classification segmentation predictions on testing data.

        Parameters
        ----------
        train_dataloader:DataLoader
                Train data.
        test_dataloader:DataLoader
                Test data.

        Returns
        -------
        array
                Predictions.

        """
        return self.fit(train_dataloader)[0].predict(test_dataloader)

    def return_model(self):
        """Returns pytorch model.
        """
        return self.model


In [30]:
# import fire
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets as Datasets
import sys
import copy
import os
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
# from models import generate_model, ModelTrainer
from PIL import Image
from pathflowai.utils import load_sql_df
import torch.nn as nn
import kornia.augmentation as K, kornia.geometry.transform as G
import kornia
from kornia.augmentation import AugmentationBase

class RandRotate(AugmentationBase):
    def __init__(self, return_transform: bool = False, max_angle: float = 90) -> None:
        super(RandRotate, self).__init__(return_transform)
        self.angle=max_angle

    def generate_parameters(self, input_shape: torch.Size):
        angles_rad = (torch.rand(input_shape[0])-0.5) * self.angle * np.pi/180.
        angles_deg = kornia.rad2deg(angles_rad)
        return dict(angles=angles_deg)

    def compute_transformation(self, input, params):
        B, _, H, W = input.shape
        angles= params['angles'].type_as(input)
        center = torch.tensor([[W / 2, H / 2]] * B).type_as(input)
        transform = G.get_rotation_matrix2d(
            center, angles, torch.ones_like(angles))
        self.cos=torch.abs(transform[0, 0, 0])
        self.sin=torch.abs(transform[0, 0, 1])
        self.center=center
        return transform

    def apply_transform(self, input, params):
        _, _, H, W = input.shape
        transform = self.compute_transformation(input, params)
        nW = int((H * self.sin) + (W * self.cos))
        nH = int((H * self.cos) + (W * self.sin))
        transform[0, 0, 2] += (nW / 2) - self.center[0,0]
        transform[0, 1, 2] += (nH / 2) - self.center[0,1]
        output = 1.-G.warp_affine(1.-input, transform, (nH, nW))
        return (output, transform)[0]

class Reshape(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,x):
        return x.view(x.shape[0],-1)

class NPYDataset(Dataset):
    def __init__(self, patch_info, npy_file, transform):
        self.ID=os.path.basename(npy_file).split('.')[0]
        self.patch_info=patch_info.loc[patch_info["ID"]==self.ID].reset_index()
        self.X=np.load(npy_file)
        self.to_pil=lambda x: Image.fromarray(x)
        self.transform=transform

    def __getitem__(self,i):
        x,y,patch_size=self.patch_info.loc[i,["x","y","patch_size"]]
        return self.transform(self.to_pil(self.X[x:x+patch_size,y:y+patch_size]))

    def __len__(self):
        return self.patch_info.shape[0]

    def embed(self,model,batch_size,out_dir):
        Z=[]
        dataloader=DataLoader(self,batch_size=batch_size,shuffle=False)
        n_batches=len(self)//batch_size
        with torch.no_grad():
            for i,X in enumerate(dataloader):
                if torch.cuda.is_available():
                    X=X.cuda()
                z=model(X).detach().cpu().numpy()
                Z.append(z)
                print(f"Processed batch {i}/{n_batches}")
        Z=np.vstack(Z)
        torch.save(dict(embeddings=Z,patch_info=self.patch_info),os.path.join(out_dir,f"{self.ID}.pkl"))
        print("Embeddings saved")
        quit()

class PandasDataset(Dataset):
    def __init__(self,df_data):
        self.df=df_data#copy.deepcopy(df)
        self.y=df_data['complete'].map(lambda x: x.numpy()).values

    def __getitem__(self,i):
        _,y,X=self.df.iloc[i]
        return X,y

    def __len__(self):
        return self.df.shape[0]

def generate_transformers(image_size=224, resize=256, mean=[], std=[], include_jitter=False):

    train_transform = transforms.Compose([
        transforms.Resize(resize)]
        + ([transforms.ColorJitter(brightness=0.4, contrast=0.4,
                                   saturation=0.4, hue=0.1)] if include_jitter else [])
        + [transforms.RandomHorizontalFlip(p=0.5),
           transforms.RandomVerticalFlip(p=0.5),
           transforms.RandomRotation(90),
           transforms.RandomResizedCrop(image_size),
           transforms.ToTensor(),
           transforms.Normalize(mean if mean else [0.5, 0.5, 0.5],
                                std if std else [0.1, 0.1, 0.1])
           ])
    val_transform = transforms.Compose([
        transforms.Resize(resize),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean if mean else [0.5, 0.5, 0.5],
                             std if std else [0.1, 0.1, 0.1])
    ])
    normalization_transform = transforms.Compose([transforms.Resize(256),
                                                  transforms.CenterCrop(
                                                      image_size),
                                                  transforms.ToTensor()])
    return {'train': train_transform, 'val': val_transform, 'test': val_transform, 'norm': normalization_transform}

def generate_kornia_transforms(image_size=224, resize=256, mean=[], std=[], include_jitter=False):
    mean=torch.tensor(mean) if mean else torch.tensor([0.5, 0.5, 0.5])
    std=torch.tensor(std) if std else torch.tensor([0.1, 0.1, 0.1])
    if torch.cuda.is_available():
        mean=mean.cuda()
        std=std.cuda()
    train_transforms=[RandRotate(max_angle=30.)]#
    if include_jitter:
        train_transforms.append(K.ColorJitter(brightness=0.4, contrast=0.4,
                                   saturation=0.4, hue=0.1))
    train_transforms.extend([
            K.RandomHorizontalFlip(p=0.5),
            K.RandomVerticalFlip(p=0.5),
#            K.RandomRotation(90),
#            K.RandomResizedCrop((image_size,image_size)),
            K.Normalize(mean,std)
           ])
    val_transforms=[
#         G.Resize((resize,resize)),
#            K.CenterCrop((image_size,image_size)),
           K.Normalize(mean,std)
           ]
    transforms=dict(train=nn.Sequential(*train_transforms),
                val=nn.Sequential(*val_transforms))
    if torch.cuda.is_available():
        for k in transforms:
            transforms[k]=transforms[k].cuda()
    return transforms

def train_model(inputs_dir='inputs_training',
                learning_rate=1e-4,
                n_epochs=300,
                crop_size=224,
                resize=256,
                mean=[0.5, 0.5, 0.5],
                std=[0.1, 0.1, 0.1],
                num_classes=2,
                architecture='resnet50',
                predict=False,
                model_save_loc='saved_model.pkl',
                predictions_save_path='predictions.pkl',
                predict_set='test',
                verbose=False,
                class_balance=True,
                extract_embeddings="",
                extract_embeddings_df="",
                embedding_out_dir="./",
                gpu_id=0,
                checkpoints_dir="checkpoints",
                accumulation=1,
                port=4444
                ):
    batch_size=1
    tensor_dataset=True
    if extract_embeddings: assert predict, "Must be in prediction mode to extract embeddings"
    torch.cuda.set_device(gpu_id)
    transformers=generate_transformers if not tensor_dataset else generate_kornia_transforms
    transformers = transformers(
        image_size=crop_size, resize=resize, mean=mean, std=std)
    if not extract_embeddings:
        if tensor_dataset:
            datasets = {x: PandasDataset(data[x]) for x in ['train','val']}#torch.load(os.path.join(inputs_dir,f"{x}_data.pth"))
        else:
            datasets = {x: Datasets.ImageFolder(os.path.join(
                inputs_dir, x), transformers[x]) for x in ['train', 'val', 'test']}

        dataloaders = {x: DataLoader(
            datasets[x], batch_size=batch_size, shuffle=(x == 'train')) for x in datasets}

#     model = generate_model(architecture,
#                            num_classes)
    model=FullyConvolutionalResnet18(num_classes=num_classes,pretrained=False)

    if torch.cuda.is_available():
        model = model.cuda()

    optimizer_opts = dict(name='adam',
                          lr=learning_rate,
                          weight_decay=1e-4)

    scheduler_opts = dict(scheduler='warm_restarts',
                          lr_scheduler_decay=0.5,
                          T_max=10,
                          eta_min=5e-8,
                          T_mult=2)

    trainer = ModelTrainer(model,
                           n_epochs,
                           None if predict else dataloaders['val'],
                           optimizer_opts,
                           scheduler_opts,
                           loss_fn='ce',
                           checkpoints_dir=checkpoints_dir,
                           tensor_dataset=tensor_dataset,
                           transforms=transformers,
                           accumulation=accumulation,
                           port=port)

    if not predict:
        print(np.unique(datasets['train'].y))
        if class_balance:
            trainer.add_class_balance_loss(datasets['train'].y)#datasets['train'].targets if not tensor_dataset else datasets['train'].tensors[1].numpy()

        trainer, min_val_loss, best_epoch=trainer.fit(dataloaders['train'],verbose=verbose)

        torch.save(trainer.model.state_dict(), model_save_loc)

    else:
        assert not tensor_dataset, "Only ImageFolder and NPYDatasets allowed"

        trainer.model.load_state_dict(torch.load(model_save_loc))

        if extract_embeddings and extract_embeddings_df:
            trainer.model=nn.Sequential(trainer.model.features,Reshape())
            patch_info=load_sql_df(extract_embeddings_df,resize)
            dataset=NPYDataset(patch_info,extract_embeddings,transformers["test"])
            dataset.embed(trainer.model,batch_size,embedding_out_dir)
            exit()

        Y = dict()

        Y['pred'],Y['true'] = trainer.predict(dataloaders[predict_set])

        # Y['true'] = datasets[predict_set].targets

        torch.save(Y, predictions_save_path)


# if __name__ == '__main__':
#     fire.Fire(train_model)


In [31]:
train_model(inputs_dir='inputs_training',
                learning_rate=1e-2,
                n_epochs=300,
                crop_size=224,
                resize=256,
                mean=[0.55, 0.5, 0.55],
                std=[0.15, 0.15, 0.15],
                num_classes=2,
                predict=False,
                model_save_loc='saved_model.pkl',
                predictions_save_path='predictions.pkl',
                predict_set='test',
                verbose=True,
                class_balance=True,
                extract_embeddings="",
                extract_embeddings_df="",
                embedding_out_dir="./",
                gpu_id=0,
                checkpoints_dir="checkpoints",
                accumulation=16,
                port=4444)#64

[0 1]
Weights: [1.2541806  0.83148559]
Epoch 0[0/750] Time:0.249, Train Loss:0.0041647134348750114
Epoch 0[1/750] Time:0.075, Train Loss:43.93566131591797
Epoch 0[2/750] Time:0.068, Train Loss:3.0040289857424796e-05
Epoch 0[3/750] Time:0.182, Train Loss:1.6212332411669195e-05
Epoch 0[4/750] Time:0.078, Train Loss:12.938154220581055
Epoch 0[5/750] Time:0.031, Train Loss:14.658008575439453
Epoch 0[6/750] Time:0.097, Train Loss:18.98540496826172
Epoch 0[7/750] Time:0.072, Train Loss:8.077390670776367
Epoch 0[8/750] Time:0.084, Train Loss:4.0531076592742465e-06
Epoch 0[9/750] Time:0.022, Train Loss:30.24176025390625
Epoch 0[10/750] Time:0.032, Train Loss:0.0
Epoch 0[11/750] Time:0.07, Train Loss:0.0
Epoch 0[12/750] Time:0.073, Train Loss:0.0
Epoch 0[13/750] Time:0.202, Train Loss:0.0
Epoch 0[14/750] Time:0.097, Train Loss:5.459548473358154
Epoch 0[15/750] Time:0.028, Train Loss:1.678027868270874
Epoch 0[16/750] Time:0.018, Train Loss:1129.12109375
Epoch 0[17/750] Time:0.107, Train Loss:244

Epoch 0[166/750] Time:0.247, Train Loss:0.0
Epoch 0[167/750] Time:0.087, Train Loss:0.0
Epoch 0[168/750] Time:0.027, Train Loss:75.25152587890625
Epoch 0[169/750] Time:0.073, Train Loss:9.2939453125
Epoch 0[170/750] Time:0.028, Train Loss:3.547647476196289
Epoch 0[171/750] Time:0.027, Train Loss:55.24066162109375
Epoch 0[172/750] Time:0.072, Train Loss:0.0
Epoch 0[173/750] Time:0.086, Train Loss:0.018440943211317062
Epoch 0[174/750] Time:0.22, Train Loss:62.35565185546875
Epoch 0[175/750] Time:0.087, Train Loss:0.0
Epoch 0[176/750] Time:0.029, Train Loss:0.030003685504198074
Epoch 0[177/750] Time:0.054, Train Loss:0.0
Epoch 0[178/750] Time:0.08, Train Loss:0.0
Epoch 0[179/750] Time:0.145, Train Loss:121.10382843017578
Epoch 0[180/750] Time:0.07, Train Loss:0.0
Epoch 0[181/750] Time:0.029, Train Loss:1.6927575416048057e-05
Epoch 0[182/750] Time:0.081, Train Loss:0.0
Epoch 0[183/750] Time:0.072, Train Loss:33.49420166015625
Epoch 0[184/750] Time:0.023, Train Loss:68.89022827148438
Epoch 

Epoch 0[331/750] Time:0.06, Train Loss:0.0
Epoch 0[332/750] Time:0.02, Train Loss:0.0
Epoch 0[333/750] Time:0.101, Train Loss:0.0
Epoch 0[334/750] Time:0.031, Train Loss:0.025420721620321274
Epoch 0[335/750] Time:0.033, Train Loss:0.0
Epoch 0[336/750] Time:0.078, Train Loss:47.758087158203125
Epoch 0[337/750] Time:0.116, Train Loss:0.0
Epoch 0[338/750] Time:0.021, Train Loss:0.0
Epoch 0[339/750] Time:0.026, Train Loss:0.0
Epoch 0[340/750] Time:0.109, Train Loss:0.0
Epoch 0[341/750] Time:0.032, Train Loss:0.0
Epoch 0[342/750] Time:0.035, Train Loss:1.1920928244535389e-07
Epoch 0[343/750] Time:0.091, Train Loss:0.0
Epoch 0[344/750] Time:0.033, Train Loss:2.2375223636627197
Epoch 0[345/750] Time:0.036, Train Loss:0.0006976078147999942
Epoch 0[346/750] Time:0.179, Train Loss:0.0
Epoch 0[347/750] Time:0.103, Train Loss:9.957414627075195
Epoch 0[348/750] Time:0.026, Train Loss:10.30575942993164
Epoch 0[349/750] Time:0.022, Train Loss:0.0
Epoch 0[350/750] Time:0.061, Train Loss:15.52175903320

Epoch 0[490/750] Time:0.122, Train Loss:0.0
Epoch 0[491/750] Time:0.112, Train Loss:0.0
Epoch 0[492/750] Time:0.112, Train Loss:1.0728830375228426e-06
Epoch 0[493/750] Time:0.054, Train Loss:36.109039306640625
Epoch 0[494/750] Time:0.081, Train Loss:0.0
Epoch 0[495/750] Time:0.034, Train Loss:4.410734163684538e-06
Epoch 0[496/750] Time:0.032, Train Loss:0.0
Epoch 0[497/750] Time:0.086, Train Loss:5.588677406311035
Epoch 0[498/750] Time:0.029, Train Loss:59.85610580444336
Epoch 0[499/750] Time:0.063, Train Loss:0.0
Epoch 0[500/750] Time:0.901, Train Loss:0.0
Epoch 0[501/750] Time:1.085, Train Loss:13.537262916564941
Epoch 0[502/750] Time:0.205, Train Loss:0.0
Epoch 0[503/750] Time:0.162, Train Loss:0.0
Epoch 0[504/750] Time:0.084, Train Loss:0.0
Epoch 0[505/750] Time:0.029, Train Loss:0.0
Epoch 0[506/750] Time:0.031, Train Loss:4.936860084533691
Epoch 0[507/750] Time:0.03, Train Loss:0.08984766155481339
Epoch 0[508/750] Time:0.06, Train Loss:0.0
Epoch 0[509/750] Time:0.022, Train Loss:0

Epoch 0[648/750] Time:0.062, Train Loss:6.308645725250244
Epoch 0[649/750] Time:0.071, Train Loss:0.0
Epoch 0[650/750] Time:0.175, Train Loss:0.0
Epoch 0[651/750] Time:0.068, Train Loss:0.0
Epoch 0[652/750] Time:0.071, Train Loss:0.0001530530134914443
Epoch 0[653/750] Time:0.155, Train Loss:0.0
Epoch 0[654/750] Time:0.062, Train Loss:0.0
Epoch 0[655/750] Time:0.113, Train Loss:2.062299427052494e-05
Epoch 0[656/750] Time:0.025, Train Loss:18.39825439453125
Epoch 0[657/750] Time:0.068, Train Loss:0.03383755311369896
Epoch 0[658/750] Time:0.054, Train Loss:0.0
Epoch 0[659/750] Time:0.024, Train Loss:0.0
Epoch 0[660/750] Time:0.075, Train Loss:3.1859259605407715
Epoch 0[661/750] Time:0.07, Train Loss:2.9878952503204346
Epoch 0[662/750] Time:0.024, Train Loss:8.271068572998047
Epoch 0[663/750] Time:0.022, Train Loss:0.0
Epoch 0[664/750] Time:0.015, Train Loss:236.17620849609375
Epoch 0[665/750] Time:0.076, Train Loss:0.0
Epoch 0[666/750] Time:0.128, Train Loss:6.7176103591918945
Epoch 0[667

...

Epoch 4[265/322] Val Loss:0.22211477160453796
Epoch 4[266/322] Val Loss:0.0002489972102921456
Epoch 4[267/322] Val Loss:1.0728830375228426e-06
Epoch 4[268/322] Val Loss:2.9858956336975098
Epoch 4[269/322] Val Loss:0.09424672275781631
Epoch 4[270/322] Val Loss:4.87473201751709
Epoch 4[271/322] Val Loss:0.004727258812636137
Epoch 4[272/322] Val Loss:0.929397702217102
Epoch 4[273/322] Val Loss:3.8299038410186768
Epoch 4[274/322] Val Loss:0.2172159105539322
Epoch 4[275/322] Val Loss:0.008313927799463272
Epoch 4[276/322] Val Loss:0.02587386593222618
Epoch 4[277/322] Val Loss:0.057166460901498795
Epoch 4[278/322] Val Loss:32.32463073730469
Epoch 4[279/322] Val Loss:3.9049603939056396
Epoch 4[280/322] Val Loss:0.07299625873565674
Epoch 4[281/322] Val Loss:24.47503662109375
Epoch 4[282/322] Val Loss:27.16602325439453
Epoch 4[283/322] Val Loss:4.442093372344971
Epoch 4[284/322] Val Loss:29.90283203125
Epoch 4[285/322] Val Loss:0.20603948831558228
Epoch 4[286/322] Val Loss:0.20320191979408264
Ep

Epoch 5[100/750] Time:0.076, Train Loss:0.15234851837158203
Epoch 5[101/750] Time:0.076, Train Loss:0.02584935538470745
Epoch 5[102/750] Time:0.844, Train Loss:0.00010847457451745868
Epoch 5[103/750] Time:0.063, Train Loss:2.3841855067985307e-07
Epoch 5[104/750] Time:0.021, Train Loss:0.0
Epoch 5[105/750] Time:0.014, Train Loss:0.0
Epoch 5[106/750] Time:0.062, Train Loss:0.0
Epoch 5[107/750] Time:0.064, Train Loss:1.0728830375228426e-06
Epoch 5[108/750] Time:0.114, Train Loss:2.3841830625315197e-06
Epoch 5[109/750] Time:0.048, Train Loss:3.814689989667386e-06
Epoch 5[110/750] Time:0.017, Train Loss:0.0
Epoch 5[111/750] Time:0.057, Train Loss:0.0
Epoch 5[112/750] Time:0.02, Train Loss:4.7801782784517854e-05
Epoch 5[113/750] Time:0.028, Train Loss:0.0
Epoch 5[114/750] Time:0.111, Train Loss:3.194758028257638e-05
Epoch 5[115/750] Time:0.023, Train Loss:0.0
Epoch 5[116/750] Time:0.034, Train Loss:2.9802276912960224e-06
Epoch 5[117/750] Time:0.027, Train Loss:80.23234558105469
Epoch 5[118/7

Epoch 5[244/750] Time:0.09, Train Loss:17.546066284179688
Epoch 5[245/750] Time:0.071, Train Loss:9.536738616588991e-07
Epoch 5[246/750] Time:0.059, Train Loss:9.059865078597795e-06
Epoch 5[247/750] Time:0.024, Train Loss:4.18088436126709
Epoch 5[248/750] Time:0.024, Train Loss:0.02939869649708271
Epoch 5[249/750] Time:0.026, Train Loss:0.0
Epoch 5[250/750] Time:0.109, Train Loss:0.0
Epoch 5[251/750] Time:0.028, Train Loss:1.1920928244535389e-07
Epoch 5[252/750] Time:0.07, Train Loss:1.0609570381348021e-05
Epoch 5[253/750] Time:0.022, Train Loss:5.376194530981593e-05
Epoch 5[254/750] Time:0.053, Train Loss:1.6689286894688848e-06
Epoch 5[255/750] Time:0.18, Train Loss:0.0
Epoch 5[256/750] Time:0.194, Train Loss:12.60132122039795
Epoch 5[257/750] Time:0.026, Train Loss:0.048702582716941833
Epoch 5[258/750] Time:0.227, Train Loss:0.0
Epoch 5[259/750] Time:0.027, Train Loss:5.8053239627042785e-05
Epoch 5[260/750] Time:0.095, Train Loss:14.154939651489258
Epoch 5[261/750] Time:0.124, Train 

Epoch 5[390/750] Time:0.014, Train Loss:0.0042199138551950455
Epoch 5[391/750] Time:0.092, Train Loss:7.390948667307384e-06
Epoch 5[392/750] Time:0.029, Train Loss:0.0
Epoch 5[393/750] Time:0.079, Train Loss:6.264097213745117
Epoch 5[394/750] Time:0.182, Train Loss:0.0011744751827791333
Epoch 5[395/750] Time:0.021, Train Loss:1.1920927533992653e-07
Epoch 5[396/750] Time:0.031, Train Loss:0.007788882590830326
Epoch 5[397/750] Time:0.035, Train Loss:7.986990567587782e-06
Epoch 5[398/750] Time:0.034, Train Loss:0.0
Epoch 5[399/750] Time:0.039, Train Loss:0.0
Epoch 5[400/750] Time:0.231, Train Loss:0.0010363452602177858
Epoch 5[401/750] Time:0.05, Train Loss:9.962113380432129
Epoch 5[402/750] Time:0.066, Train Loss:22.617523193359375
Epoch 5[403/750] Time:0.11, Train Loss:14.663041114807129
Epoch 5[404/750] Time:0.018, Train Loss:0.0018900641007348895
Epoch 5[405/750] Time:0.032, Train Loss:3.197953224182129
Epoch 5[406/750] Time:0.033, Train Loss:6.854299135738984e-05
Epoch 5[407/750] Tim

KeyboardInterrupt: 


CommandNotFoundError: Your shell has not been properly configured to use 'conda deactivate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




ERROR:visdom:Connection is already closed.
ERROR:visdom:Connection is already closed.
ERROR:visdom:Connection is already closed.
ERROR:visdom:Connection is already closed.
ERROR:visdom:Connection is already closed.
ERROR:visdom:Connection is already closed.
ERROR:visdom:Connection is already closed.
ERROR:visdom:Connection is already closed.
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refu

ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connect

ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connect

ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connect

...

ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connect

ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connect

ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connect

ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connect

ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connection refused
ERROR:visdom:[Errno 111] Connect

/usr/bin/c++
